In [5]:
import tensorflow as tf
import numpy as np
import pandas as pd
import random
import sys

In [6]:
df=pd.read_csv('train.csv')

In [7]:
text="".join(df['text'].dropna().astype(str)).lower()
print(f'Total characters in text:{len(text)}')

Total characters in text:35680894


In [8]:
print(text[:50])

berlin (reuters) - none of the german parties invo


In [9]:
vocab=sorted(set(text))
print(f'Vocabulary size:{len(vocab)}')

Vocabulary size:104


In [10]:
print(vocab[:104])

['\t', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '\xa0', '\xad', '¯', '°', '´', '¿', 'à', 'á', 'â', 'ç', 'é', 'ê', 'í', 'ñ', 'ó', 'ö', 'ü', 'ā', 'ō', '\u200a', '\u200b', '\u200e', '\u200f', '–', '—', '‘', '’', '“', '”', '•', '…', '☑', '➡', 'ツ', '️']


In [11]:
chartoindx={c:i for i,c in enumerate(vocab)}
indxtochar=np.array(vocab)
textasint=np.array([chartoindx[c]for c in text])

In [12]:
print(chartoindx)

{'\t': 0, ' ': 1, '!': 2, '"': 3, '#': 4, '$': 5, '%': 6, '&': 7, "'": 8, '(': 9, ')': 10, '*': 11, '+': 12, ',': 13, '-': 14, '.': 15, '/': 16, '0': 17, '1': 18, '2': 19, '3': 20, '4': 21, '5': 22, '6': 23, '7': 24, '8': 25, '9': 26, ':': 27, ';': 28, '<': 29, '=': 30, '>': 31, '?': 32, '@': 33, '[': 34, '\\': 35, ']': 36, '^': 37, '_': 38, '`': 39, 'a': 40, 'b': 41, 'c': 42, 'd': 43, 'e': 44, 'f': 45, 'g': 46, 'h': 47, 'i': 48, 'j': 49, 'k': 50, 'l': 51, 'm': 52, 'n': 53, 'o': 54, 'p': 55, 'q': 56, 'r': 57, 's': 58, 't': 59, 'u': 60, 'v': 61, 'w': 62, 'x': 63, 'y': 64, 'z': 65, '{': 66, '|': 67, '}': 68, '\xa0': 69, '\xad': 70, '¯': 71, '°': 72, '´': 73, '¿': 74, 'à': 75, 'á': 76, 'â': 77, 'ç': 78, 'é': 79, 'ê': 80, 'í': 81, 'ñ': 82, 'ó': 83, 'ö': 84, 'ü': 85, 'ā': 86, 'ō': 87, '\u200a': 88, '\u200b': 89, '\u200e': 90, '\u200f': 91, '–': 92, '—': 93, '‘': 94, '’': 95, '“': 96, '”': 97, '•': 98, '…': 99, '☑': 100, '➡': 101, 'ツ': 102, '️': 103}


In [13]:
print(indxtochar)

['\t' ' ' '!' '"' '#' '$' '%' '&' "'" '(' ')' '*' '+' ',' '-' '.' '/' '0'
 '1' '2' '3' '4' '5' '6' '7' '8' '9' ':' ';' '<' '=' '>' '?' '@' '[' '\\'
 ']' '^' '_' '`' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n'
 'o' 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z' '{' '|' '}' '\xa0' '\xad'
 '¯' '°' '´' '¿' 'à' 'á' 'â' 'ç' 'é' 'ê' 'í' 'ñ' 'ó' 'ö' 'ü' 'ā' 'ō'
 '\u200a' '\u200b' '\u200e' '\u200f' '–' '—' '‘' '’' '“' '”' '•' '…' '☑'
 '➡' 'ツ' '️']


In [14]:
print(textasint)

[41 44 57 ... 15 97  1]


In [15]:
seq_length=100
char_dataset=tf.data.Dataset.from_tensor_slices(textasint)
sequences=char_dataset.batch(seq_length + 1,drop_remainder=True)

In [16]:
def split_input(chunk):
  return chunk[:-1],chunk[1:]

dataset=sequences.map(split_input)

BATCH_SIZE=64
BUFFER_SIZE=10000

dataset=dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE,drop_remainder=True)

In [17]:
vocab_size=len(vocab)
embedding_dim=64
rnn_units=128

model=tf.keras.Sequential([tf.keras.layers.Embedding(vocab_size,embedding_dim,input_shape=(None,)),
                           tf.keras.layers.LSTM(rnn_units,return_sequences=True,recurrent_initializer='glorot_uniform'),
                           tf.keras.layers.Dense(vocab_size)])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [18]:
def loss(labels,logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels,logits,from_logits=True)

In [19]:
model.compile(optimizer='adam',loss=loss)

print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, None, 64)       │         6,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, None, 128)      │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, None, 104)      │        13,416 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 118,888 (464.41 KB)

 Trainable params: 118,888 (464.41 KB)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
EPOCHS=20
history=model.fit(dataset,epochs=EPOCHS)

Epoch 1/20
 713/5519 ━━━━━━━━━━━━━━━━━━━━ 12:21 154ms/step - loss: 2.5667

In [ ]:
def generate_text(model, start_string, num_generate=100, temperature=1.0):
    input_eval = [chartoindx.get(s, 0) for s in start_string.lower()]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    model.layers[1].reset_states()

    for _ in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0) / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])

    return start_string + ''.join(text_generated)

print(generate_text(model, start_string="The ", num_generate=200, temperature=0.8))